# Using Tools


In [16]:
import requests
import json
import instructor
from openai import OpenAI
from pydantic import BaseModel, Field

#Only for Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()

#--------------------------------------------------------
# Initialize the OpenAI client for Ollama   
# And add instructor wrapper for Pydantic validation
#--------------------------------------------------------
client = OpenAI(
    base_url="http://localhost:11434/v1/", 
    api_key=""
)
client = instructor.patch(client)

#--------------------------------------------------------
# Tool response model 
#--------------------------------------------------------
class WeatherResponse(BaseModel):
    temperature: float = Field(description="The current temperature in celsius for the given location.")
    response: str = Field(description="A natural language response to the user's question.")

#----------------------------------------------------------
# Tool 
#----------------------------------------------------------
def get_weather(latitude, longitude):
    """This is a publically available API that returns the weather for a given location."""   
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast"
        f"?latitude={latitude}&longitude={longitude}"
        f"&current=temperature_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]

#--------------------------------------------------------
# Tool schema
#--------------------------------------------------------
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for provided coordinates in celsius.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

In [14]:
model="llama3.1:8b"
messages=[            
    {"role": "system",  "content": "You are a helpful weather assistant."},
    {"role": "user",    "content": "What's the weather like now in Tainan, Taiwan now ?"},
]

#---------------------------------------------------------------------------------------------
# 這是 OpenAI API 官方標準的端點 ===> 用於創建聊天對話的補全 (chat completion)。
#---------------------------------------------------------------------------------------------
# 1. 根據 messages 和 tools，模型決定是要直接生成文字回覆，還是要呼叫工具 (tool call)。
#    當模型決定呼叫工具時，它的回覆會包含工具名稱和參數，但不會執行工具。
#---------------------------------------------------------------------------------------------
completion2 = client.chat.completions.create(
    model=model,    
    messages=messages,     
    temperature=0, 
    tools=tools,      
)
#print(f"\nchat completion ===>\n", json.dumps(completion2.model_dump(), indent=2))   

#----------------------------------------------------------------------------
# 2. 如果 LLM 說要 Call Tool
#----------------------------------------------------------------------------
tool_calls = completion2.choices[0].message.tool_calls
if tool_calls:  
    for tool_call in completion2.choices[0].message.tool_calls:
        # 3. 取出 Tool 的 name 和 參數  
        # 4. 把模型的回覆加入到 messages 列表中。使 messages 現在包含了使用者發問和模型的回覆  
        name = tool_call.function.name    
        args = json.loads(tool_call.function.arguments)       
        messages.append(completion2.choices[0].message)    
        print(f"\nTool ===> {name} with args: {args}, \nmessage ===> ", completion2.choices[0].message) 
                          
        # 5. Tool Calling
        result = None
        if name == "get_weather":
            result = get_weather(**args)           
            messages.append(
                {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
            )  
            print(f"\nTool Calling 結果 ===> {result}")   


Tool ===> get_weather with args: {'latitude': 22.98, 'longitude': 120.21}, 
message ===>  ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_44kppyir', function=Function(arguments='{"latitude":22.98,"longitude":120.21}', name='get_weather'), type='function', index=0)])

Tool Calling 結果 ===> {'time': '2025-09-06T11:15', 'interval': 900, 'temperature_2m': 28.4, 'wind_speed_10m': 3.7}


In [13]:
#----------------------------------------------------------------------------
# 6. 最後讓 LLM 根據工具回傳的內容，生成你想要的自然語言回應。
#    而且回傳格式會符合你定義的 WeatherResponse Pydantic schema
#----------------------------------------------------------------------------
completion = client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    tools=tools,
    response_format=WeatherResponse,
)

final_response = completion.choices[0].message.parsed
print("\nWeatherResponse Model 定義的 temperature ===> ", final_response.temperature)
print("WeatherResponse Model 定義的 response ===> ",final_response.response)



WeatherResponse Model 定義的 temperature ===>  28.4
WeatherResponse Model 定義的 response ===>  The current weather in Tainan, Taiwan is warm with a comfortable temperature of 28.4°C (82.1°F). The winds are relatively calm, blowing at approximately 3.7 meters per second.


🧑‍💻 使用者提問
     ↓
🧠 Step 1: client.chat.completions.create(...)
     └─ LLM 判斷需要用 tool，例如 get_weather
     └─ 產生 tool_call 指令 + arguments
     ↓
📨 Step 2: ✨ messages.append(模型提出的 tool_call message)
     ↓
⚙️ Step 3: 程式實際執行對應的 Tool function（如 get_weather(**args)）
     ↓
📨 Step 4: ✨ messages.append({
         role: "tool",
         tool_call_id: tool_call.id,
         content: 工具回傳結果（如天氣資料）
     })
     ↓
🧠 Step 5: client.beta.chat.completions.parse(...)
     └─ 模型根據完整上下文（使用者提問 + tool_call + 工具結果）
     └─ 產生最終自然語言回答 & 結構化資料（符合 Pydantic schema）
